# Oppmerksomhetsmekanismer og transformere

En stor ulempe med rekurrente nettverk er at alle ord i en sekvens har samme påvirkning på resultatet. Dette fører til suboptimal ytelse med standard LSTM-encoder-decoder-modeller for sekvens-til-sekvens-oppgaver, som for eksempel navngjenkjenning og maskinoversettelse. I virkeligheten har spesifikke ord i inngangssekvensen ofte større påvirkning på sekvensielle utganger enn andre.

Tenk på en sekvens-til-sekvens-modell, som maskinoversettelse. Den implementeres ved hjelp av to rekurrente nettverk, der ett nettverk (**encoder**) komprimerer inngangssekvensen til en skjult tilstand, og et annet nettverk, **decoder**, ruller ut denne skjulte tilstanden til et oversatt resultat. Problemet med denne tilnærmingen er at nettverkets endelige tilstand har vanskeligheter med å huske begynnelsen av en setning, noe som fører til dårlig modellkvalitet for lange setninger.

**Oppmerksomhetsmekanismer** gir en måte å vekte den kontekstuelle påvirkningen av hver inngangsvektor på hver utgangsprediksjon i RNN. Dette implementeres ved å lage snarveier mellom mellomliggende tilstander i inngangs-RNN og utgangs-RNN. På denne måten, når vi genererer utgangssymbolet $y_t$, tar vi hensyn til alle skjulte inngangstilstander $h_i$, med forskjellige vektkoeffisienter $\alpha_{t,i}$. 

![Bilde som viser en encoder/decoder-modell med et additivt oppmerksomhetslag](../../../../../translated_images/no/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder-modellen med additiv oppmerksomhetsmekanisme i [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), sitert fra [denne bloggposten](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Oppmerksomhetsmatrisen $\{\alpha_{i,j}\}$ representerer graden til hvilken visse inngangsord spiller en rolle i genereringen av et gitt ord i utgangssekvensen. Nedenfor er et eksempel på en slik matrise:

![Bilde som viser et eksempel på justering funnet av RNNsearch-50, hentet fra Bahdanau - arviz.org](../../../../../translated_images/no/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figur hentet fra [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Oppmerksomhetsmekanismer er ansvarlige for mye av dagens eller nesten dagens toppytelse innen naturlig språkbehandling. Å legge til oppmerksomhet øker imidlertid antallet modellparametere betydelig, noe som førte til skaleringsproblemer med RNN-er. En viktig begrensning ved skalering av RNN-er er at modellens rekurrente natur gjør det utfordrende å batch-prosessere og parallellisere treningen. I en RNN må hvert element i en sekvens behandles i sekvensiell rekkefølge, noe som betyr at det ikke enkelt kan parallelliseres.

Bruken av oppmerksomhetsmekanismer kombinert med denne begrensningen førte til opprettelsen av de nåværende toppmoderne transformermodellene vi kjenner og bruker i dag, fra BERT til OpenGPT3.

## Transformermodeller

I stedet for å sende konteksten fra hver tidligere prediksjon videre til neste evalueringssteg, bruker **transformermodeller** **posisjonelle kodinger** og **oppmerksomhet** for å fange konteksten til en gitt inngang innenfor et gitt tekstvindu. Bildet nedenfor viser hvordan posisjonelle kodinger med oppmerksomhet kan fange kontekst innenfor et gitt vindu.

![Animasjon som viser hvordan evalueringene utføres i transformermodeller.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

Siden hver inngangsposisjon kartlegges uavhengig til hver utgangsposisjon, kan transformere parallellisere bedre enn RNN-er, noe som muliggjør mye større og mer uttrykksfulle språkmodeller. Hver oppmerksomhetshode kan brukes til å lære forskjellige relasjoner mellom ord som forbedrer oppgaver innen naturlig språkbehandling.

## Bygge en enkel transformermodell

Keras inneholder ikke et innebygd Transformer-lag, men vi kan bygge vårt eget. Som før vil vi fokusere på tekstklassifisering av AG News-datasettet, men det er verdt å nevne at transformermodeller gir best resultater på mer krevende NLP-oppgaver.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nye lag i Keras bør arve `Layer`-klassen og implementere `call`-metoden. La oss starte med **Positional Embedding**-laget. Vi vil bruke [noe kode fra den offisielle Keras-dokumentasjonen](https://keras.io/examples/nlp/text_classification_with_transformer/). Vi vil anta at vi fyller alle inngangssekvenser til lengde `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Dette laget består av to `Embedding`-lag: ett for å embedde tokens (på en måte vi har diskutert tidligere) og ett for tokenposisjoner. Tokenposisjoner opprettes som en sekvens av naturlige tall fra 0 til `maxlen` ved hjelp av `tf.range`, og sendes deretter gjennom embed-laget. De to resulterende embedding-vektorene legges sammen, og gir en posisjonsbasert representasjon av input med formen `maxlen`$\times$`embed_dim`.

Nå skal vi implementere transformer-blokken. Den vil ta utdataene fra det tidligere definerte embed-laget:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Nå er vi klare til å definere den komplette transformer-modellen:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer-modeller

**BERT** (Bidirectional Encoder Representations from Transformers) er et svært stort flerlags transformatornettverk med 12 lag for *BERT-base* og 24 for *BERT-large*. Modellen blir først forhåndstrent på en stor mengde tekstdata (Wikipedia + bøker) ved hjelp av usupervisert trening (forutsi maskerte ord i en setning). Under forhåndstreningen absorberer modellen et betydelig nivå av språkforståelse som deretter kan utnyttes med andre datasett ved hjelp av finjustering. Denne prosessen kalles **transfer learning**.

![bilde fra http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/no/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Det finnes mange varianter av transformatorarkitekturer, inkludert BERT, DistilBERT, BigBird, OpenGPT3 og flere, som kan finjusteres.

La oss se hvordan vi kan bruke en forhåndstrent BERT-modell for å løse vårt tradisjonelle sekvensklassifiseringsproblem. Vi vil låne ideen og noe kode fra [offisiell dokumentasjon](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

For å laste inn forhåndstrente modeller, vil vi bruke **Tensorflow hub**. Først, la oss laste inn den BERT-spesifikke vektorisatoren:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Det er viktig at du bruker den samme vektoriseringen som det opprinnelige nettverket ble trent med. I tillegg returnerer BERT-vektoriseringen tre komponenter:
* `input_word_ids`, som er en sekvens av token-numre for inndata-setningen
* `input_mask`, som viser hvilken del av sekvensen som inneholder faktisk inndata, og hvilken del som er utfylling. Dette ligner på masken som produseres av `Masking`-laget
* `input_type_ids` brukes for oppgaver innen språklig modellering, og gjør det mulig å spesifisere to inndata-setninger i én sekvens.

Deretter kan vi opprette en BERT-funksjonsekstraktor:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Så BERT-laget returnerer en rekke nyttige resultater:
* `pooled_output` er et resultat av å gjennomsnittliggjøre alle tokens i sekvensen. Du kan se på det som en intelligent semantisk representasjon av hele nettverket. Det tilsvarer utdataene fra `GlobalAveragePooling1D`-laget i vår forrige modell.
* `sequence_output` er utdataene fra det siste transformer-laget (tilsvarer utdataene fra `TransformerBlock` i modellen vår ovenfor).
* `encoder_outputs` er utdataene fra alle transformer-lagene. Siden vi har lastet inn en 4-lags BERT-modell (som du sikkert kan gjette fra navnet, som inneholder `4_H`), har den 4 tensorer. Den siste er den samme som `sequence_output`.

Nå skal vi definere den ende-til-ende klassifiseringsmodellen. Vi vil bruke *funksjonell modelldefinisjon*, der vi definerer modellens input og deretter gir en serie uttrykk for å beregne dens output. Vi vil også gjøre vektene til BERT-modellen ikke-trainable og kun trene den siste klassifikatoren:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Selv om det er få trenbare parametere, er prosessen ganske treg, fordi BERT-funksjonsekstraktoren er beregningsmessig tung. Det ser ut til at vi ikke klarte å oppnå rimelig nøyaktighet, enten på grunn av mangel på trening eller mangel på modellparametere.

La oss prøve å låse opp BERT-vektene og trene den også. Dette krever en veldig liten læringsrate, og en mer forsiktig treningsstrategi med **warmup**, ved bruk av **AdamW**-optimalisatoren. Vi vil bruke `tf-models-official`-pakken for å opprette optimalisatoren:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Som du kan se, går treningen ganske sakte – men du kan eksperimentere og trene modellen i noen få epoker (5-10) for å se om du kan oppnå et bedre resultat sammenlignet med tilnærmingene vi har brukt tidligere.

## Huggingface Transformers-bibliotek

En annen veldig vanlig (og litt enklere) måte å bruke Transformer-modeller på er [HuggingFace-pakken](https://github.com/huggingface/), som tilbyr enkle byggeklosser for ulike NLP-oppgaver. Den er tilgjengelig både for Tensorflow og PyTorch, et annet veldig populært rammeverk for nevrale nettverk.

> **Merk**: Hvis du ikke er interessert i å se hvordan Transformers-biblioteket fungerer, kan du hoppe til slutten av denne notatboken, fordi du ikke vil se noe vesentlig annerledes enn det vi har gjort ovenfor. Vi vil gjenta de samme trinnene for å trene BERT-modellen ved å bruke et annet bibliotek og en vesentlig større modell. Dermed innebærer prosessen en del langvarig trening, så du kan bare velge å se gjennom koden.

La oss se hvordan problemet vårt kan løses ved hjelp av [Huggingface Transformers](http://huggingface.co).


Det første vi må gjøre er å velge modellen vi skal bruke. I tillegg til noen innebygde modeller, har Huggingface et [nettbasert modellbibliotek](https://huggingface.co/models), hvor du kan finne mange flere forhåndstrente modeller fra fellesskapet. Alle disse modellene kan lastes inn og brukes bare ved å oppgi et modellnavn. Alle nødvendige binærfiler for modellen vil automatisk bli lastet ned.

Noen ganger vil du kanskje måtte laste inn dine egne modeller. I så fall kan du spesifisere katalogen som inneholder alle relevante filer, inkludert parametere for tokenizer, `config.json`-filen med modellparametere, binære vekter osv.

Fra modellnavnet kan vi opprette både modellen og tokenizer. La oss starte med en tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objektet inneholder `encode`-funksjonen som kan brukes direkte til å kode tekst:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Vi kan også bruke tokenizer til å kode en sekvens på en måte som er egnet for å sende til modellen, dvs. inkludert `token_ids`, `input_mask`-felt, osv. Vi kan også spesifisere at vi ønsker Tensorflow-tensorer ved å gi argumentet `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

I vårt tilfelle vil vi bruke den forhåndstrente BERT-modellen kalt `bert-base-uncased`. *Uncased* indikerer at modellen er ikke-følsom for store og små bokstaver.

Når vi trener modellen, må vi gi en tokenisert sekvens som input, og derfor vil vi designe en databehandlingspipeline. Siden `tokenizer.encode` er en Python-funksjon, vil vi bruke samme tilnærming som i forrige enhet ved å kalle den med `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Nå kan vi laste inn den faktiske modellen ved å bruke `BertForSequenceClassification`-pakken. Dette sikrer at modellen vår allerede har en nødvendig arkitektur for klassifisering, inkludert den endelige klassifikatoren. Du vil se en advarsel som sier at vektene til den endelige klassifikatoren ikke er initialisert, og at modellen vil kreve forhåndstrening - det er helt greit, fordi det er akkurat det vi skal gjøre!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Som du kan se fra `summary()`, inneholder modellen nesten 110 millioner parametere! Antakeligvis, hvis vi ønsker en enkel klassifiseringsoppgave på et relativt lite datasett, ønsker vi ikke å trene BERT-baselaget:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nå er vi klare til å begynne treningen!

> **Note**: Å trene en fullskala BERT-modell kan være svært tidkrevende! Derfor vil vi kun trene den for de første 32 batchene. Dette er bare for å vise hvordan modelltreningen settes opp. Hvis du er interessert i å prøve fullskala trening - fjern bare parameterne `steps_per_epoch` og `validation_steps`, og gjør deg klar til å vente!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Hvis du øker antall iterasjoner og venter lenge nok, og trener i flere epoker, kan du forvente at BERT-klassifisering gir oss den beste nøyaktigheten! Dette er fordi BERT allerede forstår språkets struktur ganske godt, og vi trenger bare å finjustere den endelige klassifiseringen. Men siden BERT er en stor modell, tar hele treningsprosessen lang tid og krever betydelig datakraft! (GPU, og helst mer enn én).

> **Note:** I vårt eksempel har vi brukt en av de minste forhåndstrente BERT-modellene. Det finnes større modeller som sannsynligvis vil gi bedre resultater.


## Viktige punkter

I denne enheten har vi sett på svært moderne modellarkitekturer basert på **transformere**. Vi har brukt dem til vår oppgave med tekstklassifisering, men på samme måte kan BERT-modeller brukes til enhetsuttrekk, spørsmål-svar og andre NLP-oppgaver.

Transformer-modeller representerer dagens toppnivå innen NLP, og i de fleste tilfeller bør de være den første løsningen du eksperimenterer med når du implementerer skreddersydde NLP-løsninger. Likevel er det svært viktig å forstå de grunnleggende prinsippene bak rekurrente nevrale nettverk som ble diskutert i denne modulen, dersom du ønsker å bygge avanserte nevrale modeller.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiske oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
